In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/disaster-tweets-data/sample_submission.csv
/kaggle/input/disaster-tweets-data/train.csv
/kaggle/input/disaster-tweets-data/test.csv


<h3>Import Libraries</h3>

In [2]:
import re
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModel
from nltk.tokenize import word_tokenize
import spacy

<h3>Get Data</h3>

In [3]:
train_data = pd.read_csv('/kaggle/input/disaster-tweets-data/train.csv')
test_data = pd.read_csv('/kaggle/input/disaster-tweets-data/test.csv')

In [4]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
test_data.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
test_data['target'] = False
data = pd.concat([train_data, test_data], sort=False)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10876 entries, 0 to 3262
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        10876 non-null  int64 
 1   keyword   10789 non-null  object
 2   location  7238 non-null   object
 3   text      10876 non-null  object
 4   target    10876 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 509.8+ KB


<h3>Cleaning Location Feature</h3>

In [7]:
data['location'] = data['location'].fillna('')

In [8]:
# lowercase text
data['location_clean'] = data['location'].str.lower()

# remove any non
data['location_clean'] = data.apply(lambda x: re.sub(r'([^\w\s])', '', x['location_clean']), axis=1)

# tokenize text
data['location_clean'] = data.apply(lambda x: word_tokenize(x['location_clean']), axis=1)
data['location'] = data.apply(lambda x: word_tokenize(x['location']), axis=1)

In [9]:
tokens_raw = sum(data['location'], [])

In [10]:
nlp = spacy.load('en_core_web_sm')
spacy_doc = nlp(' '.join(tokens_raw))

In [11]:
ner_df = pd.DataFrame(columns=['ner_tag'])

for token in spacy_doc.ents:
  if pd.isna(token.label_) is False:
      ner_df = pd.concat([ner_df, pd.DataFrame.from_records([{'ner_tag': token.label_}])], ignore_index=True)

ner_df.head()

,ner_tag
0,PERSON
1,DATE
2,GPE
3,GPE
4,PERSON


In [12]:
ner_df['ner_tag'].value_counts(ascending=False)

ner_tag
GPE            3031
ORG            1220
PERSON          678
CARDINAL        133
LOC              73
FAC              62
DATE             49
PRODUCT          26
NORP             24
MONEY            14
WORK_OF_ART      13
EVENT             6
PERCENT           5
QUANTITY          4
ORDINAL           4
TIME              3
LANGUAGE          2
Name: count, dtype: int64

In [13]:
data = pd.concat([data.reset_index(drop=True), ner_df.reset_index(drop=True)], axis=1)
data.head()

,id,keyword,location,text,target,location_clean,ner_tag
0,1,NaN,[],Our Deeds are the Reason of this #earthquake M...,1,[],PERSON
1,4,NaN,[],Forest fire near La Ronge Sask. Canada,1,[],DATE
2,5,NaN,[],All residents asked to 'shelter in place' are ...,1,[],GPE
3,6,NaN,[],"13,000 people receive #wildfires evacuation or...",1,[],GPE
4,7,NaN,[],Just got sent this photo from Ruby #Alaska as ...,1,[],PERSON


<h3>Text Preprocessing</h3>

In [14]:
nltk.download('stopwords')
en_stopwords = stopwords.words('english')
print(en_stopwords)

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# lowercase
data['text_clean'] = data['text'].str.lower()

# remove links from the text
data['text_clean'] = data.apply(lambda x: re.sub(r'http\S+|www\.\S+', '', x['text_clean']), axis=1)

# remove any special character or non words
data['text_clean'] = data.apply(lambda x: re.sub(r'([^\w\s])', "", x['text_clean']), axis=1)

# remove stopwords
data['text_clean'] = data['text_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (en_stopwords)]))

text = data['text_clean'].tolist()
text[:5]

['deeds reason earthquake may allah forgive us',
 'forest fire near la ronge sask canada',
 'residents asked shelter place notified officers evacuation shelter place orders expected',
 '13000 people receive wildfires evacuation orders california',
 'got sent photo ruby alaska smoke wildfires pours school']

In [16]:
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the tweets
encodings = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

<h3>Scale the othe</h3>

<h3>Create Custom BERT model</h3>

<h3>Model Finetuning</h3>

<h3>Model Testing</h3>